Provo a creare una matrice alternativa:
- pazienti sulle righe
- ogni combinazione di condition-trials sulle colonne

In [23]:
import json
sys.path.append('../')
from utilities import from_dict_to_df
import pandas as pd
import itertools
from tqdm import tqdm

In [2]:
# open dataset
with open('../../data/full_data.json', 'r') as file:
    data = json.load(file)

In [3]:
conditions_df = pd.DataFrame(data['Conditions'])
therapies_df = pd.DataFrame(data['Therapies'])
patients_df = pd.DataFrame(data['Patients'])

In [32]:
prova = pd.DataFrame(columns = c, index= pat_cured)
#prova = prova.fillna(0)
prova

,"(cond_0, ther_0)","(cond_0, ther_1)","(cond_0, ther_2)","(cond_0, ther_3)","(cond_0, ther_4)","(cond_0, ther_5)","(cond_0, ther_6)","(cond_0, ther_7)","(cond_0, ther_8)","(cond_0, ther_9)",...,"(cond_308, ther_57)","(cond_308, ther_58)","(cond_308, ther_59)","(cond_308, ther_60)","(cond_308, ther_61)","(cond_308, ther_62)","(cond_308, ther_63)","(cond_308, ther_64)","(cond_308, ther_65)","(cond_308, ther_66)"
pat_14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
pat_78,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
pat_271,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
pat_322,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
pat_340,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
pat_9582,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
pat_9700,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
pat_9785,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
pat_9878,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
prova.columns[1]

('cond_0', 'ther_1')

In [11]:
conditions_df.id.values
therapies_df.id.values

c = list(itertools.product(conditions_df.id.values, therapies_df.id.values))
len(c)

20703

In [145]:
def utility_matrix():
    # # open dataset
    # with open('../../data/full_data.json', 'r') as file:
    #     data = json.load(file)
    
    conditions_df = pd.DataFrame(data['Conditions'])
    therapies_df = pd.DataFrame(data['Therapies'])
    patients_df = pd.DataFrame(data['Patients'])
    
    # initialize empty utility matrix
    c = list(itertools.product(conditions_df.id.values, therapies_df.id.values))
    prova = pd.DataFrame(columns = c, index= pat_cured)
    
    for p in tqdm(pat_cured, desc="Filling utility matrix"):
        # extract data for the patient
        pat = patients_df[patients_df.id == p]

        # trials and cond
        trials = pd.DataFrame(pat.trials.values[0])
        cond = pd.DataFrame(pat.conditions.values[0])
        
        # add true cond_id to trial data
        trials['cond_id'] = ""

        for idx,row in trials.iterrows():
            pat_cond = row['condition']
            cond_id = cond[cond.id == pat_cond].kind.values[0]
            trials.loc[idx,'cond_id'] = cond_id
            
        # fill utility matrix
        for idx,row in trials.iterrows():
            matrix_pat = prova
            
            cond = row['cond_id']
            ther = row['therapy']
            succ = row['successful']
            col = (cond, ther)
            matrix_pat.at[p, col] = succ
        # normalize
        norm_matrix = matrix_pat.sub(matrix_pat.mean(axis=1), axis=0)
        final_matrix = norm_matrix.fillna(0)
    return final_matrix

In [143]:
def utility_matrix_single_p(patients_id):
    # # open dataset
    # with open('../../data/full_data.json', 'r') as file:
    #     data = json.load(file)
    
    conditions_df = pd.DataFrame(data['Conditions'])
    therapies_df = pd.DataFrame(data['Therapies'])
    patients_df = pd.DataFrame(data['Patients'])
    
    # initialize empty utility matrix
    c = list(itertools.product(conditions_df.id.values, therapies_df.id.values))
    prova = pd.DataFrame(columns = c, index= list([patients_id]))
    
    # extract data for the patient
    pat = patients_df[patients_df.id == patients_id]

    # trials and cond
    trials = pd.DataFrame(pat.trials.values[0])
    cond = pd.DataFrame(pat.conditions.values[0])
    
    # add true cond_id to trial data
    trials['cond_id'] = ""

    for idx,row in trials.iterrows():
        pat_cond = row['condition']
        cond_id = cond[cond.id == pat_cond].kind.values[0]
        trials.loc[idx,'cond_id'] = cond_id
        
    # fill utility matrix
    for idx,row in trials.iterrows():
        matrix_pat = prova
        
        cond = row['cond_id']
        ther = row['therapy']
        succ = row['successful']
        col = (cond, ther)
        matrix_pat.at[patients_id, col] = succ
    # normalize
    norm_matrix = matrix_pat.sub(matrix_pat.mean(axis=1), axis=0)
    final_matrix = norm_matrix.fillna(0)
    return final_matrix

In [26]:
pat_cured = []
for i in patients_df.id.values:
    if any(d['kind'] == 'cond_14' and d['cured'] != None for d in patients_df[patients_df.id == i].conditions.values[0]):
        #print(i)
        
        pat_cured.append(i)

In [146]:
u_mat = utility_matrix()

Filling utility matrix: 100%|██████████| 108/108 [03:11<00:00,  1.77s/it]


In [164]:
cos_sim = []
for p in pat_cured:
    pat_row = u_mat[u_mat.index == p]
    vect = pat_row.to_numpy()
    cos = 1 - distance.cosine(vect_0, vect)
    cos_sim.append(cos)

In [165]:
cos_sim

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.03973325322155363,
 0.0,
 0.020497668846309658,
 0.00036031904961997263,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.03652859857837398,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 -0.017567145417449526,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 -0.0006886054703083566,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 -0.0008600003427203795,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 -0.0009642468321808817,
 0.0,
 0.0,
 0.010045470597916495,
 0.0,
 0.0,
 0.0,
 0.0,
 -0.00020531215095820343,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 -0.007413388963727119,
 0.0,
 0.0,
 0.0]

In [157]:
pi = u_mat[u_mat.index == "pat_14"]
vect = pi.to_numpy()

20703

In [158]:
mat_0 = utility_matrix_single_p("pat_0")
vect_0 = mat_0.to_numpy()

In [160]:
mat_0 # 20767


,"(cond_0, ther_0)","(cond_0, ther_1)","(cond_0, ther_2)","(cond_0, ther_3)","(cond_0, ther_4)","(cond_0, ther_5)","(cond_0, ther_6)","(cond_0, ther_7)","(cond_0, ther_8)","(cond_0, ther_9)",...,"(cond_308, ther_57)","(cond_308, ther_58)","(cond_308, ther_59)","(cond_308, ther_60)","(cond_308, ther_61)","(cond_308, ther_62)","(cond_308, ther_63)","(cond_308, ther_64)","(cond_308, ther_65)","(cond_308, ther_66)"
pat_0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [161]:
pi # 25607 

,"(cond_0, ther_0)","(cond_0, ther_1)","(cond_0, ther_2)","(cond_0, ther_3)","(cond_0, ther_4)","(cond_0, ther_5)","(cond_0, ther_6)","(cond_0, ther_7)","(cond_0, ther_8)","(cond_0, ther_9)",...,"(cond_308, ther_57)","(cond_308, ther_58)","(cond_308, ther_59)","(cond_308, ther_60)","(cond_308, ther_61)","(cond_308, ther_62)","(cond_308, ther_63)","(cond_308, ther_64)","(cond_308, ther_65)","(cond_308, ther_66)"
pat_14,0,0,0.0,0.0,0,0,0,0,0,0.0,...,0,0.0,0,0,0.0,0.0,0.0,0.0,0,0.0


In [162]:
import numpy as np
import scipy.spatial.distance as distance
c = 1 - distance.cosine(vect_0, vect)
c

0.0

In [176]:
import random
NUM_OF_CONDITIONS = random.sample(range(1,12), 1)[0]
NUM_OF_CONDITIONS
ages = []
for n in range(0,10000):
    age = random.sample(range(18,99), 1)[0]
    ages.append(age)
    
ages

[79,
 52,
 22,
 57,
 34,
 90,
 92,
 92,
 68,
 34,
 44,
 54,
 77,
 31,
 80,
 65,
 22,
 98,
 56,
 82,
 51,
 94,
 22,
 98,
 56,
 57,
 43,
 67,
 93,
 24,
 52,
 57,
 91,
 54,
 71,
 51,
 86,
 69,
 22,
 73,
 62,
 69,
 98,
 25,
 34,
 30,
 77,
 58,
 74,
 71,
 54,
 79,
 96,
 26,
 97,
 50,
 68,
 60,
 41,
 39,
 68,
 61,
 59,
 53,
 45,
 36,
 41,
 87,
 42,
 44,
 87,
 40,
 87,
 20,
 80,
 89,
 66,
 46,
 98,
 34,
 25,
 36,
 35,
 19,
 52,
 33,
 76,
 63,
 37,
 97,
 49,
 40,
 73,
 23,
 78,
 85,
 68,
 85,
 50,
 58,
 36,
 22,
 38,
 53,
 80,
 90,
 97,
 66,
 22,
 35,
 89,
 62,
 22,
 19,
 76,
 75,
 58,
 73,
 25,
 24,
 71,
 66,
 19,
 89,
 56,
 42,
 58,
 78,
 46,
 59,
 67,
 47,
 87,
 70,
 58,
 43,
 89,
 63,
 22,
 75,
 64,
 26,
 20,
 62,
 69,
 40,
 29,
 83,
 69,
 68,
 81,
 58,
 79,
 47,
 55,
 24,
 66,
 93,
 29,
 92,
 84,
 66,
 47,
 37,
 56,
 92,
 46,
 44,
 94,
 51,
 54,
 26,
 34,
 85,
 56,
 95,
 31,
 88,
 22,
 18,
 23,
 46,
 80,
 83,
 90,
 70,
 96,
 86,
 94,
 30,
 37,
 75,
 59,
 91,
 25,
 54,
 75,
 80,
 37,
 61,
